In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

response = requests.get("https://www.lemonde.fr/en/")
doc = BeautifulSoup(response.text)

In [3]:
items = doc.select('.article')

In [8]:
rows = []

for item in items:
    row = {}

    # Headline
    try:
        row['headline'] = (item.select_one('.article__title-label') or item.select_one('.article__title')).text.strip()
    except:
        row['headline'] = 'no headline'

    # Description
    try:
        row['description'] = item.select_one('.article__desc').text.strip()
    except:
        row['description'] = 'no description'

    # URL
    try:
        row['url'] = item.select_one("a[href]").get("href")
    except:
        try:
            row['url'] = item.get('href')
        except:
            row['url'] = None

    # Byline
    try:
        if item.select_one('.article__byline'):
            row['byline'] = item.select_one('.article__byline').text.strip()
        elif item.select_one('.article__author-name'):
            row['byline'] = item.select_one('.article__author-name').text.strip()
        else:
            row['byline'] = None
    except:
        row['byline'] = None

    # Premium status
    try:
        if item.find(class_='icon__premium').text.strip() == "Subscribers only":
            row['premium'] = True
    except:
        row['premium'] = False

    # Image URL
    try:
        row['image_url'] = item.find('img').get('src') or item.find('img').get('data-src')
    except:
        row['image_url'] = None

    # Article Type
    try:
        row['article type'] = item.find(class_='article__type').text.strip()
    except:
        row['article type'] = None

    rows.append(row)


In [10]:
len(rows)

53

In [12]:
df = pd.json_normalize(rows)

In [14]:
df.to_csv('le_monde.csv')